In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
from reservoirnetwork.network import ReservoirNetwork

In [17]:
resnet = ReservoirNetwork(
    ((0, 1, 2, 2), (0, 0, -1, 1)), 
    links=[(1, 0), (2, 1), (3, 1)]
)
resnet

<xarray.Dataset>
Dimensions:        (node: 4, link: 3, Two: 2)
Coordinates:
  * node           (node) int64 0 1 2 3
Dimensions without coordinates: link, Two
Data variables:
    mesh           <U1 'a'
    y_of_node      (node) float64 0.0 1.0 2.0 2.0
    x_of_node      (node) float64 0.0 0.0 -1.0 1.0
    nodes_at_link  (link, Two) int64 0 1 2 1 1 3

<xarray.Dataset>
Dimensions:        (node: 4, link: 3, Two: 2)
Coordinates:
  * node           (node) int64 0 1 2 3
Dimensions without coordinates: link, Two
Data variables:
    mesh           <U1 'a'
    y_of_node      (node) float64 0.0 1.0 2.0 2.0
    x_of_node      (node) float64 0.0 0.0 -1.0 1.0
    nodes_at_link  (link, Two) int64 0 1 2 1 1 3